# Chapter 3: SLAM Fundamentals - Demo Notebook

This notebook demonstrates a basic RGB-D SLAM system using the TUM dataset.

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
import sys

# Add src to path
sys.path.insert(0, str(Path.cwd().parent.parent))

from src.datasets import TUMDatasetFetcher
from src.slam import RGBDSlam
from src.utils import plot_trajectory

%matplotlib inline

## Step 1: Load the TUM RGB-D Dataset

We'll use the `fr1/desk` sequence, which shows a handheld camera moving around an office desk.

In [ ]:
# Initialize dataset fetcher
fetcher = TUMDatasetFetcher('../../data')
sequence = 'fr1/desk'

# Download if needed (will skip if already downloaded)
seq_path = fetcher.download_sequence(sequence)
print(f"Dataset path: {seq_path}")

# Load data
timestamps, rgb_paths, depth_paths = fetcher.load_associations(sequence)
gt_timestamps, gt_poses = fetcher.load_poses(sequence)
K = fetcher.get_camera_intrinsics(sequence)

print(f"Found {len(rgb_paths)} RGB-D pairs")
print(f"Camera intrinsics:\n{K}")

## Step 2: Visualize Sample Data

In [ ]:
# Load a sample frame
idx = 100
rgb = cv2.imread(rgb_paths[idx])
rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
depth = cv2.imread(depth_paths[idx], cv2.IMREAD_UNCHANGED)
depth_meters = depth.astype(np.float32) / 5000.0

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].imshow(rgb)
axes[0].set_title('RGB Image', fontsize=14)
axes[0].axis('off')

depth_vis = depth_meters.copy()
depth_vis[depth_vis == 0] = np.nan
im = axes[1].imshow(depth_vis, cmap='plasma', vmin=0, vmax=4.0)
axes[1].set_title('Depth Map', fontsize=14)
axes[1].axis('off')
plt.colorbar(im, ax=axes[1], fraction=0.046, label='Depth (m)')

plt.tight_layout()
plt.show()

## Step 3: Initialize SLAM System

In [ ]:
# Initialize SLAM
slam = RGBDSlam(
    camera_matrix=K,
    detector_type='ORB',
    max_features=1000,
    loop_closure_threshold=0.5
)

print("SLAM system initialized")

## Step 4: Run SLAM

In [ ]:
# Process sequence
estimated_poses = slam.process_sequence(rgb_paths, depth_paths, max_frames=200)

print(f"\nProcessed {len(estimated_poses)} frames")
stats = slam.get_statistics()
for key, value in stats.items():
    print(f"  {key}: {value}")

## Step 5: Visualize Results

In [ ]:
# Match ground truth
gt_poses_matched = []
for i in range(len(estimated_poses)):
    ts = timestamps[i]
    idx = np.argmin(np.abs(gt_timestamps - ts))
    gt_poses_matched.append(gt_poses[idx])

gt_poses_matched = np.array(gt_poses_matched)

# Plot
plot_trajectory(estimated_poses, gt_poses_matched, "SLAM Trajectory")
plt.show()